In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load dataset
df = pd.read_csv("../Data/Fraud Original Full.csv")

# Display basic info
print(df.info())
print(df.describe())

# Check for missing values
print(f"Missing Values:\n{df.isnull().sum()}")

# Count occurrences of each fraud class
fraud_counts = df["fraud"].value_counts()

# Display results
print(fraud_counts)

# Compute correlation with fraud variable
correlation_matrix = df.corr()

# Sort correlations with fraud in descending order
fraud_correlation = correlation_matrix["fraud"].sort_values(ascending=False)

# Display results
print(fraud_correlation)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498121 entries, 0 to 498120
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   trustLevel                 498121 non-null  int64  
 1   totalScanTimeInSeconds     498121 non-null  int64  
 2   grandTotal                 498121 non-null  float64
 3   lineItemVoids              498121 non-null  int64  
 4   scansWithoutRegistration   498121 non-null  int64  
 5   quantityModification       498121 non-null  int64  
 6   scannedLineItemsPerSecond  498121 non-null  float64
 7   valuePerSecond             498121 non-null  float64
 8   lineItemVoidsPerPosition   498121 non-null  float64
 9   fraud                      498121 non-null  int64  
dtypes: float64(4), int64(6)
memory usage: 38.0 MB
None
          trustLevel  totalScanTimeInSeconds     grandTotal  lineItemVoids  \
count  498121.000000           498121.000000  498121.000000  498121.000